# Who is a Playmaker?

According to [Wikipedia](https://en.wikipedia.org/wiki/Playmaker), a playmaker is a player who controls the flow of the team's offensive play, and is often involved in **passing moves which lead to goals**, through their **vision**, **technique**, **ball control**, **creativity**, and **passing ability**.

According to [Squawka](https://www.squawka.com/en/what-is-a-deep-lying-playmaker-footballs-best-examples/), playmaking is not just about defensive midfielders who can pass, nor is it about the kind of central midfielders who will drop deep to receive the ball and then also push up, or go wide, etc. This is about footballers who play primarily at the base of midfield, and whose chief function is to dictate play from there with intelligent passes through and beyond opposing midfield lines.

In [ ]:
from IPython.display import Image
Image("../pics/Playmakers.png")

# Import Required Libraries

In [ ]:
# Data Manipulation libraries:
import numpy as np
import pandas as pd
from copy import deepcopy

# Plotting libraries
import mplsoccer
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.projections import get_projection_class
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
pd.set_option("display.max_columns", 50)

# Load the Data

**Data source** --> Statsbomb \
**Competition** --> La Liga \
**Season** --> 2019-20

In [ ]:
eventPassDataLaLiga2019 = pd.read_csv("../data/passDataLaLiga2020.csv",
                                      low_memory=False)

In [ ]:
passRelDataLaLiga2019 = pd.read_csv("../data/passRelDataLaLiga2020.csv",
                                    low_memory=False)

# Exploring Traits of a Playmaker

1. Vision
1. Ball control
1. Creativity
1. Passing ability

## Vision

### Get Vision Data

In [ ]:
eventPassDataLaLiga2019.columns.values

In [ ]:
eventPassDataLaLiga2019.groupby(["player.id"]).agg({"player.name": "first",
                                                    "team.name": "first",
                                                    "type.id": "count",
                                                    "pass.through_ball": "sum",
                                                    "pass.shot_assist": "sum",
                                                    "pass.goal_assist": "sum"})

In [ ]:
playerWisedf = eventPassDataLaLiga2019.groupby(["player.id"]).agg({"player.name": "first",
                                                                   "team.name": "first",
                                                                   "type.id": "count",
                                                                   "pass.through_ball": "sum",
                                                                   "pass.shot_assist": "sum",
                                                                   "pass.goal_assist": "sum"})

In [ ]:
playerMatchMinsdf = eventPassDataLaLiga2019.drop_duplicates(subset=["player.id", "match_id"]).groupby(["player.id"])\
.agg({"minsPlayed": "sum"})

In [ ]:
playerWisedf = pd.concat([playerWisedf, playerMatchMinsdf], axis=1)

In [ ]:
playerWisedf

In [ ]:
playerWisedf.rename(columns={"type.id": "totalPasses",
                             "pass.through_ball": "passTB",
                             "pass.shot_assist": "passSA",
                             "pass.goal_assist": "passGA"},
                    inplace=True)

### Calculate Per90

In [ ]:
per90Cols = ["totalPasses", "passTB", "passSA", "passGA"]

In [ ]:
playerWisedf.columns

In [ ]:
for col in per90Cols:
    playerWisedf[col + "Per90"] = playerWisedf[col].divide(playerWisedf["minsPlayed"]).multiply(90)

In [ ]:
playerWisedf.head()

### Vision Ratings

Using Standardized weights:

visionRating =  passTBPer90 * 0.2 + passSAPer90 * 0.3 + passGAPer90 * 0.5

In [ ]:
playerWisedf["visionRating"] =\
    (playerWisedf["passTBPer90"]*0.2)\
        .add(playerWisedf["passSAPer90"]*0.3)\
        .add(playerWisedf["passGAPer90"]*0.5)

In [ ]:
playerWisedf.head()

In [ ]:
playerWisedf["visionRating"].nlargest(10).index

In [ ]:
playerWisedf.loc[playerWisedf["visionRating"].nlargest(10).index, ["player.name", "team.name", "visionRating"]]

In [ ]:
playerWisedf.loc[3122]

In [ ]:
playerWiseFiltereddf = playerWisedf[(playerWisedf["minsPlayed"] > 45)]

In [ ]:
playerWiseFiltereddf.loc[playerWiseFiltereddf["visionRating"].nlargest(10).index]\
[["player.name", "team.name", "visionRating"]]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler((1, 10))
playerWiseFiltereddf["visionRating"] =\
    scaler.fit_transform(np.array(playerWiseFiltereddf["visionRating"]).reshape(-1, 1))

In [ ]:
playerWiseFiltereddf["visionRating"].max()

In [ ]:
playerWiseFiltereddf.loc[playerWiseFiltereddf["visionRating"].nlargest(10).index]\
[["player.name", "team.name", "visionRating"]]

## Ball Control

1. Carry
1. Dispossessed
1. Dribble
1. Miscontrol

In [ ]:
eventsDataLaLiga2019 = pd.read_csv("../data/matchwise_events_data_updated.csv")

In [ ]:
eventsDataLaLiga2019[["type.id", "type.name"]].drop_duplicates()

### Data Prep - Carry

In [ ]:
eventsDataLaLiga2019[eventsDataLaLiga2019["type.id"] == 43]

In [ ]:
carryDataLaLiga1920 = eventsDataLaLiga2019[eventsDataLaLiga2019["type.id"] == 43]

In [ ]:
carryDataLaLiga1920.groupby(["player.id"]).agg({"player.name": "first",
                                                    "team.name": "first",
                                                    "type.id": "count"})\
.sort_values("type.id", ascending=False)

In [ ]:
carryPlayerData = carryDataLaLiga1920.groupby(["player.id"]).agg({"player.name": "first",
                                                                  "team.name": "first",
                                                                  "type.id": "count"})

In [ ]:
carryPlayerData.rename(columns={"type.id": "totCarry"}, inplace=True)

In [ ]:
pd.concat([carryPlayerData, playerMatchMinsdf], axis=1)

In [ ]:
pd.merge(carryPlayerData, playerMatchMinsdf, how="left", left_index=True, right_index=True)

In [ ]:
carryPlayerData = pd.merge(carryPlayerData, playerMatchMinsdf, how="left",
                           left_index=True, right_index=True)

In [ ]:
carryPlayerData["totCarryPer90"] = carryPlayerData["totCarry"].divide(carryPlayerData["minsPlayed"])*90

In [ ]:
carryPlayerData.head()

### Data Prep - Dispossessed

In [ ]:
eventsDataLaLiga2019[eventsDataLaLiga2019["type.id"] == 3].head()

In [ ]:
eventsDataLaLiga2019[(eventsDataLaLiga2019["type.id"] == 3)
                     & (eventsDataLaLiga2019["player.id"].isnull())]

In [ ]:
dispDataLaLiga1920 = eventsDataLaLiga2019[eventsDataLaLiga2019["type.id"] == 3]

In [ ]:
dispDataLaLiga1920.groupby(["player.id"]).agg({"player.name": "first",
                                               "team.name": "first",
                                               "type.id": "count"})\
.sort_values("type.id", ascending=False)

In [ ]:
dispPlayerData = dispDataLaLiga1920.groupby(["player.id"]).agg({"player.name": "first",
                                                                  "team.name": "first",
                                                                  "type.id": "count"})

In [ ]:
dispPlayerData.rename(columns={"type.id": "totDisp"}, inplace=True)

In [ ]:
pd.concat([dispPlayerData, playerMatchMinsdf], axis=1)

In [ ]:
pd.merge(dispPlayerData, playerMatchMinsdf, how="left", left_index=True, right_index=True)

In [ ]:
dispPlayerData = pd.merge(dispPlayerData, playerMatchMinsdf, how="left",
                          left_index=True, right_index=True)

In [ ]:
dispPlayerData["totDispPer90"] = dispPlayerData["totDisp"].divide(dispPlayerData["minsPlayed"])*90

In [ ]:
carryPlayerData.head()

In [ ]:
dispPlayerData.drop(columns=['player.name', 'team.name', 'minsPlayed'], inplace=True)

### Data Prep - Dribble

In [ ]:
eventsDataLaLiga2019[eventsDataLaLiga2019["type.id"] == 14].head()

In [ ]:
dribbleDataLaLiga1920 = eventsDataLaLiga2019[eventsDataLaLiga2019["type.id"] == 14]

In [ ]:
dribbleDataLaLiga1920.groupby(["player.id"]).agg({"player.name": "first",
                                               "team.name": "first",
                                               "type.id": "count"})\
.sort_values("type.id", ascending=False)

In [ ]:
dribblePlayerData = dribbleDataLaLiga1920.groupby(["player.id"]).agg({"player.name": "first",
                                                                  "team.name": "first",
                                                                  "type.id": "count"})

In [ ]:
dribblePlayerData.rename(columns={"type.id": "totDribble"}, inplace=True)

In [ ]:
pd.merge(dribblePlayerData, playerMatchMinsdf, how="left", left_index=True, right_index=True)

In [ ]:
dribblePlayerData = pd.merge(dribblePlayerData, playerMatchMinsdf, how="left", left_index=True, right_index=True)

In [ ]:
dribblePlayerData["totDribblePer90"] = dribblePlayerData["totDribble"].divide(dribblePlayerData["minsPlayed"])*90

In [ ]:
dribblePlayerData.loc[dribblePlayerData["totDribblePer90"].nlargest(10).index,
                      ["player.name", "team.name", "totDribblePer90"]]

In [ ]:
dribblePlayerData.drop(columns=['player.name', 'team.name', 'minsPlayed'], inplace=True)

### Data Prep - Miscontrol

In [ ]:
eventsDataLaLiga2019[eventsDataLaLiga2019["type.id"] == 38].head()

In [ ]:
miscontrolDataLaLiga1920 = eventsDataLaLiga2019[eventsDataLaLiga2019["type.id"] == 38]

In [ ]:
miscontrolDataLaLiga1920.groupby(["player.id"]).agg({"player.name": "first",
                                               "team.name": "first",
                                               "type.id": "count"})\
.sort_values("type.id", ascending=False)

In [ ]:
miscontrolPlayerData = miscontrolDataLaLiga1920.groupby(["player.id"]).agg({"player.name": "first",
                                                                  "team.name": "first",
                                                                  "type.id": "count"})

In [ ]:
miscontrolPlayerData.rename(columns={"type.id": "totMiscontrol"}, inplace=True)

In [ ]:
pd.merge(miscontrolPlayerData, playerMatchMinsdf, how="left", left_index=True, right_index=True)

In [ ]:
miscontrolPlayerData = pd.merge(miscontrolPlayerData, playerMatchMinsdf,
                                how="left",
                                left_index=True, right_index=True)

In [ ]:
miscontrolPlayerData["totMiscontrolPer90"] =\
    miscontrolPlayerData["totMiscontrol"].divide(miscontrolPlayerData["minsPlayed"])*90

In [ ]:
miscontrolPlayerData.loc[miscontrolPlayerData["totMiscontrolPer90"].nsmallest(10).index,
                         ["player.name", "team.name", "totMiscontrolPer90"]]

In [ ]:
miscontrolPlayerData.columns

In [ ]:
miscontrolPlayerData.drop(columns=['player.name', 'team.name', 'minsPlayed'], inplace=True)

### Ball Control Ratings

In [ ]:
""" Get Ball Control Data """
ballControlData = pd.merge(carryPlayerData, dispPlayerData,
                           how="outer", left_index=True, right_index=True)
ballControlData = pd.merge(ballControlData, dribblePlayerData,
                           how="outer", left_index=True, right_index=True)
ballControlData = pd.merge(ballControlData, miscontrolPlayerData,
                           how="outer", left_index=True, right_index=True)

In [ ]:
ballControlData.info()

Using Non-Standardized weights:

bcRating = (totCarryPer90 * 2) + (totDispPer90 * -1) + (totDribblePer90 * 1) + (totMiscontrolPer90 * -2)

In [ ]:
ballControlData["bcRating"] =\
    (ballControlData["totCarryPer90"]*2)\
        .add(ballControlData["totDispPer90"]*-1)\
        .add(ballControlData["totDribblePer90"]*1)\
        .add(ballControlData["totMiscontrolPer90"]*-2)

In [ ]:
ballControlData.loc[ballControlData["bcRating"].nlargest(20).index]\
[["player.name", "team.name", "bcRating"]]

In [ ]:
scaler = MinMaxScaler((1, 10))
ballControlData["bcRating"] =\
    scaler.fit_transform(np.array(ballControlData["bcRating"]).reshape(-1, 1))

In [ ]:
ballControlData

In [ ]:
ballControlData[["totCarryPer90", "totDispPer90", "totDribblePer90", "totMiscontrolPer90"]]=\
    ballControlData[["totCarryPer90", "totDispPer90", "totDribblePer90", "totMiscontrolPer90"]].fillna(0)

In [ ]:
ballControlData.head()

In [ ]:
ballControlData["bcRating"] =\
    (ballControlData["totCarryPer90"]*2)\
        .add(ballControlData["totDispPer90"]*-1)\
        .add(ballControlData["totDribblePer90"]*1)\
        .add(ballControlData["totMiscontrolPer90"]*-2)

In [ ]:
ballControlData["bcRating"] =\
    scaler.fit_transform(np.array(ballControlData["bcRating"]).reshape(-1, 1))

In [ ]:
ballControlData.loc[ballControlData["bcRating"].nlargest(20).index]\
[["player.name", "team.name", "bcRating"]]

## Creativity

1. Carry
1. Dribble
1. Through Ball
1. Shot Assist
1. Goal Assist

### Get Creativity Data

In [ ]:
carryPlayerData.drop(columns=['player.name', 'team.name', 'minsPlayed'], inplace=True)

In [ ]:
""" Get Creativity Data """
creativityData = pd.merge(playerWisedf, dribblePlayerData,
                           how="outer", left_index=True, right_index=True)
creativityData = pd.merge(creativityData, carryPlayerData,
                           how="outer", left_index=True, right_index=True)

In [ ]:
creativityData.info()

### Creativity Ratings

Using Standardized weights:

bcRating = (totCarryPer90 * 0.05) + (passTBPer90 * 0.2) + (totDribblePer90 * 0.1)  + (passSAPer90 * 0.25) + (passGAPer90 * 0.4)

In [ ]:
creativityData["creativityRating"] =\
    (creativityData["totCarryPer90"]*0.05)\
        .add(creativityData["passTBPer90"]*0.2)\
        .add(creativityData["totDribblePer90"]*0.1)\
        .add(creativityData["passSAPer90"]*0.25)\
        .add(creativityData["passGAPer90"]*0.4)

In [ ]:
creativityData.loc[creativityData["creativityRating"].nlargest(10).index]\
[["player.name", "team.name", "creativityRating"]]

In [ ]:
creativityData["creativityRating"] =\
    scaler.fit_transform(np.array(creativityData["creativityRating"]).reshape(-1, 1))

In [ ]:
creativityData.isnull().sum()

In [ ]:
creativityData[["totCarryPer90", "passTBPer90", "totDribblePer90", "passSAPer90", "passGAPer90"]] =\
    creativityData[["totCarryPer90", "passTBPer90", "totDribblePer90", "passSAPer90", "passGAPer90"]].fillna(0)

In [ ]:
creativityData["creativityRating"] =\
    (creativityData["totCarryPer90"]*0.05)\
        .add(creativityData["passTBPer90"]*0.2)\
        .add(creativityData["totDribblePer90"]*0.1)\
        .add(creativityData["passSAPer90"]*0.25)\
        .add(creativityData["passGAPer90"]*0.4)

In [ ]:
creativityData["creativityRating"] =\
    scaler.fit_transform(np.array(creativityData["creativityRating"]).reshape(-1, 1))

In [ ]:
creativityData.loc[creativityData["creativityRating"].nlargest(10).index]\
[["player.name", "team.name", "creativityRating"]]

## Passing Ability

1. Accuracy
1. Ground Pass Accuracy
1. Low Pass Accuracy
1. High Pass Accuracy
1. Miscommunication
1. Under Pressure Accuracy
1. Through Ball Accuracy

### Total Passing Data

In [ ]:
eventPassDataLaLiga2019.groupby(["player.id"]).agg({"player.name": "first",
                                                    "team.name": "first",
                                                    "type.id": "count",
                                                    "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
totPassData = eventPassDataLaLiga2019.groupby(["player.id"]).agg({"player.name": "first",
                                                    "team.name": "first",
                                                    "type.id": "count",
                                                    "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
totPassData.rename(columns={"type.id": "totPasses",
                            "pass.outcome.id": "succPasses"},
                  inplace=True)

In [ ]:
totPassData["passAccuracy"] = totPassData["succPasses"].divide(totPassData["totPasses"])

In [ ]:
totPassData.head()

#### Accuracy Distribution

In [ ]:
binList = []
for i in range(0, 101, 10):
    binList.append(i/100)

In [ ]:
binList

In [ ]:
[i/100 for i in range(0, 101, 10)]

In [ ]:
totPassData["passAccuracy"].hist(bins=[i/100 for i in range(0, 101, 10)])

### Ground Pass Data

In [ ]:
eventPassDataLaLiga2019[["pass.height.id", "pass.height.name"]].drop_duplicates()

In [ ]:
gpData = eventPassDataLaLiga2019[eventPassDataLaLiga2019["pass.height.id"] == 1]

In [ ]:
gpData.groupby(["player.id"]).agg({"type.id": "count",
                                   "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
gpPlayerData = gpData.groupby(["player.id"]).agg({"type.id": "count",
                                                  "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
gpPlayerData.rename(columns={"type.id": "totGPasses",
                             "pass.outcome.id": "succGPasses"},
                    inplace=True)

In [ ]:
gpPlayerData["gpAccuracy"] = gpPlayerData["succGPasses"].divide(gpPlayerData["totGPasses"])

#### Accuracy Distribution

In [ ]:
gpPlayerData["gpAccuracy"].hist(bins=[i/100 for i in range(0, 101, 10)])

### Low Pass Data

In [ ]:
lpData = eventPassDataLaLiga2019[eventPassDataLaLiga2019["pass.height.id"] == 2]

In [ ]:
lpData.groupby(["player.id"]).agg({"type.id": "count",
                                   "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
lpPlayerData = lpData.groupby(["player.id"]).agg({"type.id": "count",
                                                  "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
lpPlayerData.rename(columns={"type.id": "totLPasses",
                             "pass.outcome.id": "succLPasses"},
                    inplace=True)

In [ ]:
lpPlayerData["lpAccuracy"] = lpPlayerData["succLPasses"].divide(lpPlayerData["totLPasses"])

In [ ]:
lpPlayerData

#### Accuracy Distribution

In [ ]:
lpPlayerData["lpAccuracy"].hist(bins=[i/100 for i in range(0, 101, 10)])

### High Pass Data

In [ ]:
hpData = eventPassDataLaLiga2019[eventPassDataLaLiga2019["pass.height.id"] == 3]

In [ ]:
hpData.groupby(["player.id"]).agg({"type.id": "count",
                                   "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
hpPlayerData = hpData.groupby(["player.id"]).agg({"type.id": "count",
                                                  "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
hpPlayerData.rename(columns={"type.id": "totHPasses",
                             "pass.outcome.id": "succHPasses"},
                    inplace=True)

In [ ]:
hpPlayerData["hpAccuracy"] = hpPlayerData["succHPasses"].divide(hpPlayerData["totHPasses"])

In [ ]:
hpPlayerData

#### Accuracy Distribution

In [ ]:
hpPlayerData["hpAccuracy"].hist(bins=[i/100 for i in range(0, 101, 10)])

### Under Pressure Data

In [ ]:
eventPassDataLaLiga2019[eventPassDataLaLiga2019["under_pressure"].notnull()]

In [ ]:
upData = eventPassDataLaLiga2019[eventPassDataLaLiga2019["under_pressure"].notnull()]

In [ ]:
upData.groupby(["player.id"]).agg({"type.id": "count",
                                   "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
upPlayerData = upData.groupby(["player.id"]).agg({"type.id": "count",
                                                  "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
upPlayerData.rename(columns={"type.id": "totPassesUP",
                             "pass.outcome.id": "succPassesUP"},
                    inplace=True)

In [ ]:
upPlayerData["upAccuracy"] =\
    upPlayerData["succPassesUP"].divide(upPlayerData["totPassesUP"])

#### Accuracy Distribution

In [ ]:
upPlayerData["upAccuracy"].hist(bins=[i/100 for i in range(0, 101, 10)])

### Through Ball Data

In [ ]:
eventPassDataLaLiga2019[eventPassDataLaLiga2019["pass.through_ball"].notnull()]

In [ ]:
tbData = eventPassDataLaLiga2019[eventPassDataLaLiga2019["pass.through_ball"].notnull()]

In [ ]:
tbData.groupby(["player.id"]).agg({"type.id": "count",
                                   "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
tbPlayerData = tbData.groupby(["player.id"]).agg({"type.id": "count",
                                   "pass.outcome.id": lambda x: (x.isnull()).sum()})

In [ ]:
tbPlayerData.rename(columns={"type.id": "totPassesTB",
                             "pass.outcome.id": "succPassesTB"},
                    inplace=True)

In [ ]:
tbPlayerData["tbAccuracy"] =\
    tbPlayerData["succPassesTB"].divide(tbPlayerData["totPassesTB"])

#### Accuracy Distribution

In [ ]:
tbPlayerData["tbAccuracy"].hist(bins=[i/100 for i in range(0, 101, 10)])

In [ ]:
eventPassDataLaLiga2019[eventPassDataLaLiga2019["pass.through_ball"].notnull()]["pass.outcome.id"].unique()

### Miscommunication Data

In [ ]:
eventPassDataLaLiga2019[eventPassDataLaLiga2019["pass.miscommunication"].notnull()]

In [ ]:
len(eventPassDataLaLiga2019[eventPassDataLaLiga2019["pass.miscommunication"].notnull()])/len(eventPassDataLaLiga2019)*100

NOTE: We will reject the **Miscommunication** parameter since there are only not enough data here (there are only 0.06% passes where a miscommunication has happened)

### Passing Ability Rating

In [ ]:
""" Get Passing Ability Data """
# Join Total Passes df with Ground Passes df:
passingAbilityData = pd.merge(totPassData, gpPlayerData,
                           how="outer", left_index=True, right_index=True)

# Join Final Pass Ability df with Low Passes df:
passingAbilityData = pd.merge(passingAbilityData, lpPlayerData,
                           how="outer", left_index=True, right_index=True)

# Join Final Pass Ability df with High Passes df:
passingAbilityData = pd.merge(passingAbilityData, hpPlayerData,
                           how="outer", left_index=True, right_index=True)

# Join Final Pass Ability df with Under Pressure Passes df:
passingAbilityData = pd.merge(passingAbilityData, upPlayerData,
                           how="outer", left_index=True, right_index=True)

# Join Final Pass Ability df with Through Ball Passes df:
passingAbilityData = pd.merge(passingAbilityData, tbPlayerData,
                           how="outer", left_index=True, right_index=True)

Using Standardized weights:

paRating = (passAccuracy * 0.08) + (gpAccuracy * 0.02) + (lpAccuracy * 0.1)  + (hpAccuracy * 0.15) + (upAccuracy * 0.25) + (tbAccuracy * 0.4)

In [ ]:
passingAbilityData["paRating"] =\
    (passingAbilityData["passAccuracy"]*0.08)\
        .add(passingAbilityData["gpAccuracy"]*0.02)\
        .add(passingAbilityData["lpAccuracy"]*0.1)\
        .add(passingAbilityData["hpAccuracy"]*0.15)\
        .add(passingAbilityData["upAccuracy"]*0.25)\
        .add(passingAbilityData["tbAccuracy"]*0.4)

In [ ]:
passingAbilityData.loc[passingAbilityData["paRating"].nlargest(10).index]\
[["player.name", "team.name", "paRating"]]

In [ ]:
passingAbilityData["paRating"] =\
    scaler.fit_transform(np.array(passingAbilityData["paRating"]).reshape(-1, 1))

In [ ]:
passingAbilityData.columns

In [ ]:
passingAbilityData[['passAccuracy', 'gpAccuracy',
                    'lpAccuracy', 'hpAccuracy',
                    'upAccuracy', 'tbAccuracy']] =\
    passingAbilityData[['passAccuracy', 'gpAccuracy',
                    'lpAccuracy', 'hpAccuracy',
                    'upAccuracy', 'tbAccuracy']].fillna(0)

In [ ]:
passingAbilityData["paRating"] =\
    (passingAbilityData["passAccuracy"]*0.05)\
        .add(passingAbilityData["gpAccuracy"]*0.05)\
        .add(passingAbilityData["lpAccuracy"]*0.1)\
        .add(passingAbilityData["hpAccuracy"]*0.15)\
        .add(passingAbilityData["upAccuracy"]*0.25)\
        .add(passingAbilityData["tbAccuracy"]*0.4)

In [ ]:
passingAbilityData["paRating"] =\
    scaler.fit_transform(np.array(passingAbilityData["paRating"]).reshape(-1, 1))

In [ ]:
passingAbilityData.loc[passingAbilityData["paRating"].nlargest(10).index]\
[["player.name", "team.name", "paRating"]]

# Playmaker Ratings

In [ ]:
playerWisedf.info()

In [ ]:
playerWisedf.columns

In [ ]:
""" Get Playmaker Data """

# Join Vision  df with Ball Control df:
playmakerDataFinal = pd.merge(playerWisedf[['player.name', 'team.name',
                                            'minsPlayed', 'visionRating']],
                              ballControlData["bcRating"],
                              how="outer",
                              left_index=True, right_index=True)

# Join Final Playmaker df with Creativity df:
playmakerDataFinal = pd.merge(playmakerDataFinal,
                              creativityData["creativityRating"],
                              how="outer",
                              left_index=True, right_index=True)

# Join Final Playmaker df with Passing Ability df:
playmakerDataFinal = pd.merge(playmakerDataFinal,
                              passingAbilityData['paRating'],
                              how="outer",
                              left_index=True, right_index=True)

In [ ]:
playmakerDataFinal.head()

In [ ]:
playmakerDataFinal.columns

In [ ]:
playmakerDataFinal[['visionRating', 'bcRating',
                    'creativityRating', 'paRating']].mean(axis=1)

In [ ]:
playmakerDataFinal["playmakerRating"] =\
    playmakerDataFinal[['visionRating', 'bcRating',
                        'creativityRating', 'paRating']].mean(axis=1)

In [ ]:
playmakerDataFinal.loc[playmakerDataFinal["playmakerRating"].nlargest(10).index]\
    [["player.name", "team.name", "playmakerRating"]]

In [ ]:
playmakerDataFinalFiltered = playmakerDataFinal[playmakerDataFinal["minsPlayed"] > 45]

In [ ]:
playmakerDataFinalFiltered.loc[
    playmakerDataFinalFiltered["playmakerRating"].nlargest(10).index]\
        [["player.name", "team.name", "playmakerRating"]]

Using Standardized weights:

playmakerRating = (visionRating * 0.4) + (bcRating * 0.2) + (creativityRating * 0.3)  + (paRating * 0.1)

In [ ]:
playmakerDataFinal["playmakerRating"] =\
    (playmakerDataFinal["visionRating"]*0.4)\
        .add(playmakerDataFinal["bcRating"]*0.2)\
        .add(playmakerDataFinal["creativityRating"]*0.3)\
        .add(playmakerDataFinal["paRating"]*0.1)

In [ ]:
playmakerDataFinalFiltered = playmakerDataFinal[playmakerDataFinal["minsPlayed"] > 45]

In [ ]:
playmakerDataFinalFiltered.loc[
    playmakerDataFinalFiltered["playmakerRating"].nlargest(10).index]\
        [["player.name", "team.name", "playmakerRating"]]